## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,60.30,89,11,3.42,few clouds
1,1,Tazovskiy,RU,67.4667,78.7000,-21.41,99,19,7.05,few clouds
2,2,Yerbogachen,RU,61.2767,108.0108,5.74,97,100,2.28,overcast clouds
3,3,La Rioja,ES,42.2500,-2.5000,47.05,88,7,4.56,clear sky
4,4,Khatanga,RU,71.9667,102.5000,-38.02,100,76,2.75,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Salalah,OM,17.0151,54.0924,80.69,19,0,8.05,clear sky
13,13,Acapulco,MX,16.8634,-99.8901,84.06,61,76,1.66,broken clouds
15,15,Mucurapo,TT,10.6667,-61.5333,88.25,70,98,1.01,overcast clouds
19,19,Vao,NC,-22.6667,167.4833,76.50,87,0,11.12,clear sky
24,24,Catuday,PH,16.2923,119.8062,80.65,79,92,23.67,overcast clouds
25,25,Mandera,KE,3.9366,41.8670,87.28,28,90,8.32,overcast clouds
26,26,Rikitea,PF,-23.1203,-134.9692,76.19,77,5,23.51,light rain
31,31,Kapaa,US,22.0752,-159.3190,75.52,78,40,1.01,light rain
33,33,Atuona,PF,-9.8000,-139.0333,78.33,75,12,22.88,few clouds
34,34,Vaini,TO,-21.2000,-175.2000,76.30,69,93,17.72,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                201
City                   201
Country                199
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Salalah,OM,80.69,clear sky,17.0151,54.0924,
13,Acapulco,MX,84.06,broken clouds,16.8634,-99.8901,
15,Mucurapo,TT,88.25,overcast clouds,10.6667,-61.5333,
19,Vao,NC,76.50,clear sky,-22.6667,167.4833,
24,Catuday,PH,80.65,overcast clouds,16.2923,119.8062,
25,Mandera,KE,87.28,overcast clouds,3.9366,41.8670,
26,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,
31,Kapaa,US,75.52,light rain,22.0752,-159.3190,
33,Atuona,PF,78.33,few clouds,-9.8000,-139.0333,
34,Vaini,TO,76.30,overcast clouds,-21.2000,-175.2000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping")
        

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
# Used info from the following site to convert empty strings to NaNs:  https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(15.0, 15.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))